#### ✅ Step-by-Step: เตรียม Local ETL Project แบบงานจริง

##### 1. สร้างโฟลเดอร์โปรเจกต์

In [ ]:
  mkdir etl-sales-pipeline
  cd etl-sales-pipeline

##### 2. โครงสร้างไฟล์ที่แนะนำ (ตาม best practice)

In [ ]:
etl-sales-pipeline/
│
├── data/                  ← เก็บ raw, processed, output data
│   ├── raw/
│   ├── processed/
│   └── output/
│
├── etl/                   ← โค้ดหลักของ ETL
│   ├── extract.py         ← โหลดข้อมูลจากแหล่งต่างๆ (API, CSV)
│   ├── transform.py       ← จัดรูปแบบ, clean, enrich ข้อมูล
│   ├── load.py            ← เซฟลงไฟล์ / DB / Cloud storage
│   └── utils.py           ← ฟังก์ชันช่วย (เช่น logging, date parsing)
│
├── config/                ← พวก config file, credential, parameter
│   └── config.yaml
│
├── jobs/                  ← script ที่ใช้รัน pipeline ตามวัน
│   └── run_etl.py
│
├── notebooks/             ← สำหรับทดลองบน notebook ชั่วคราว
│   └── eda_transform.ipynb
│
├── requirements.txt       ← รายการ dependencies
├── .env                   ← เก็บความลับ (เช่น API key)
└── README.md


| โฟลเดอร์     | ใช้สำหรับ                                                                                 | ตัวอย่าง                                          |
| ------------ | ----------------------------------------------------------------------------------------- | ------------------------------------------------- |
| `raw/`       | ข้อมูลต้นฉบับที่ยังไม่แตะต้อง                                                             | raw CSV, JSON จาก API                             |
| `processed/` | ข้อมูลที่ผ่านการ clean / transform เบื้องต้น เช่น remove null, filter outliers, cast type | `book_sales_cleaned.csv`                          |
| `output/`    | ข้อมูลสุดท้ายสำหรับแสดงผล, BI, dashboard, export to report                                | `monthly_sales_summary.csv`, `top_books_2024.csv` |


##### 3. ติดตั้ง environment


In [ ]:
python -m venv venv
source venv/bin/activate      # บน Linux/macOS
# หรือ venv\Scripts\activate  # บน Windows

pip install pyspark python-dotenv pyyaml

In [ ]:
# requirements.txt
pyspark
python-dotenv
pyyaml

##### 4. สร้าง `config.yaml` ตัวอย่าง

In [ ]:
data_source:
  csv_path: "data/raw/sales_data.csv"

output:
  parquet_path: "data/output/sales_summary.parquet"

##### 5. ตัวอย่าง `extract.py`

In [ ]:
from pyspark.sql import SparkSession

def extract_csv(spark: SparkSession, path: str):
    return spark.read.option("header", True).csv(path)

##### 6. ตัวอย่าง `transform.py`

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, to_date

def clean_sales_data(df: DataFrame) -> DataFrame:
    return (
        df.withColumn("amount", col("amount").cast("float"))
          .withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
          .dropna()
    )

##### 7. ตัวอย่าง `load.py`

In [ ]:
def save_to_parquet(df, path: str):
    df.write.mode("overwrite").parquet(path)

##### 8. Script สำหรับรันงาน `jobs/run_etl.py`

In [ ]:
from pyspark.sql import SparkSession
import yaml
from etl.extract import extract_csv # # type: ignore
from etl.transform import clean_sales_data # # type: ignore
from etl.load import save_to_parquet # type: ignore

# Load config
with open("config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

spark = SparkSession.builder.appName("SalesETL").getOrCreate()

# ETL flow
df_raw = extract_csv(spark, config["data_source"]["csv_path"])
df_clean = clean_sales_data(df_raw)
save_to_parquet(df_clean, config["output"]["parquet_path"])

##### 9. ตัวอย่างการรัน pipeline

In [ ]:
python jobs/run_etl.py

##### 10.หากอยากดูผลลัพธ์ Parquet แบบเร็ว ๆ

In [ ]:
df = spark.read.parquet("data/output/sales_summary.parquet") # type: ignore
df.show()

##### 🔁 สรุปขั้นตอน

| Step | Task                                         |
| ---- | -------------------------------------------- |
| 1.   | สร้างโครงสร้างโปรเจกต์                       |
| 2.   | แยกไฟล์ extract, transform, load             |
| 3.   | ใช้ config / .env เพื่อแยก parameter         |
| 4.   | รันผ่าน script (`jobs/run_etl.py`)           |
| 5.   | ทดสอบกับ sample data จริง (CSV / JSON / API) |
